In [9]:
from pymongo import MongoClient
import pymongo
from pprint import pprint
import pandas as pd
import json
import csv
import numpy as np
pymongo.version

'4.10.1'

### WORKBOOK I SUMMARY<br>Objectives: 
1. Transform the historic election csv data into a basic, 'shallow' json file for loading into a mongo database. This is the structure of theshallow Json file.
![Layered  JSON Structure for "Shallow" Historic Elelction Data](images/shallowJson.png)<br>
The 'shallow' json corresponds to this datframe:<br>
![Dataframe corresponding to "Shallow" Json](images/dataframe.png)<br>
2. Recast the historic election csv data into a more granualar json file for loading into mongodb database. The restructured json file is orgainzed into thirty-none dictionaries, each corresponding to one election. Each election-dictionary has three levels, national, state and county.<br>
![Layered  JSON Structure for "Deep" Historic Elelction Data](images/mongoDBLayers.png)<br>
3. Write the json files to disk for subsequent uplaod to mongodb

#### FUNCTIONS

In [94]:
def default(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, np.integer):
        return int(obj)
    else:
        return super().default(obj)

### Convert csv to shallow Json File

#### -read csv file into dataframe, convert NaN to zeros, convet fips float to int

In [14]:
historicElection_df = pd.read_csv('Data/election_results.csv')
historicElection_df.drop('county_end_date', axis=1, inplace=True)
historicElection_df['fips'] = historicElection_df['fips'].fillna(0)
historicElection_df['fips'] = historicElection_df['fips'].astype(int)
historicElection_df.head()

,election_year,fips,county_name,state,dem_votes,dem_nominee,rep_votes,rep_nominee
0,1868,1001,AUTAUGA,AL,851,Horatio Seymour,1505,Ulysses S. Grant
1,1872,1001,AUTAUGA,AL,669,Horace Greeley,1593,Ulysses S. Grant
2,1876,1001,AUTAUGA,AL,804,Samuel J. Tilden,1576,Rutherford B. Hayes
3,1880,1001,AUTAUGA,AL,978,Winfield Scott Hancock,974,James A. Garfield
4,1884,1001,AUTAUGA,AL,911,Grover Cleveland,877,James G. Blaine


#### - convert to dict

In [17]:
electionDict = historicElection_df.to_dict('records')
electionDict[5]

{'election_year': 1888,
 'fips': 1001,
 'county_name': 'AUTAUGA',
 'state': 'AL',
 'dem_votes': 893,
 'dem_nominee': 'Grover Cleveland',
 'rep_votes': 519,
 'rep_nominee': 'Benjamin Harrison'}

#### -write jsonified file

In [24]:
#elections_json_object_1 = json.dumps(electionDict, default=default, indent = 4)
elections_json_object_1 = json.dumps(electionDict, indent = 4)

In [26]:
with open("Data/historic_election_mongo.json", "w") as outfile: 
    outfile.write(elections_json_object_1)

#### - check read json file back into df and compare original with recreated df (checked: OK)

In [38]:
hist_elec_from_Json_df = pd.read_json('Data/historic_election_mongo.json', orient='records')
hist_elec_from_Json_df.head()

,election_year,fips,county_name,state,dem_votes,dem_nominee,rep_votes,rep_nominee
0,1868,1001,AUTAUGA,AL,851,Horatio Seymour,1505,Ulysses S. Grant
1,1872,1001,AUTAUGA,AL,669,Horace Greeley,1593,Ulysses S. Grant
2,1876,1001,AUTAUGA,AL,804,Samuel J. Tilden,1576,Rutherford B. Hayes
3,1880,1001,AUTAUGA,AL,978,Winfield Scott Hancock,974,James A. Garfield
4,1884,1001,AUTAUGA,AL,911,Grover Cleveland,877,James G. Blaine


In [40]:
hist_elec_from_Json_df.dtypes

election_year     int64
fips              int64
county_name      object
state            object
dem_votes         int64
dem_nominee      object
rep_votes         int64
rep_nominee      object
dtype: object

In [44]:
#hist_elec_from_Json_df_list = list(hist_elec_from_Json_df)
#print(type(hist_elec_from_Json_df_list))

<class 'list'>


### Make restructered 'deep' json file for granular mongodb <br>
1. construct sub-dataframes by election year
2. calculate national total stats
3. work down through states and counties using nested loop

In [74]:
election_year_list = hist_elec_from_Json_df['election_year'].unique()
print(year_list)

[1868 1872 1876 1880 1884 1888 1892 1896 1900 1904 1908 1912 1916 1920
 1924 1928 1932 1936 1940 1944 1948 1952 1956 1960 1964 1968 1972 1976
 1980 1984 1988 1992 1996 2000 2004 2008 2012 2016 2020]


In [76]:
election_df_list = []
for year in year_list:
    one_election_df = hist_elec_from_Json_df.loc[hist_elec_from_Json_df['election_year'] == year]
    election_df_list.append(one_election_df)
on_df = election_df_list[0]
on_df.head()


,election_year,fips,county_name,state,dem_votes,dem_nominee,rep_votes,rep_nominee
0,1868,1001,AUTAUGA,AL,851,Horatio Seymour,1505,Ulysses S. Grant
39,1868,1003,BALDWIN,AL,673,Horatio Seymour,558,Ulysses S. Grant
78,1868,1005,BARBOUR,AL,2210,Horatio Seymour,3168,Ulysses S. Grant
117,1868,1007,BIBB,AL,744,Horatio Seymour,492,Ulysses S. Grant
156,1868,1009,BLOUNT,AL,461,Horatio Seymour,275,Ulysses S. Grant


In [115]:
elections={} # will hold individual election dictionaries
for single_election_df in election_df_list:
    tmp_election_dict = {}
    state_name=''
    county_name=''
    election_year=''
    #vote_summary_df = getTotalNationalVote(single_election_df)
    nationalVoteTotalDem = single_election_df['dem_votes'].sum()
    nationalVoteTotalRep = single_election_df['rep_votes'].sum()
    
    election_year                                 = int((single_election_df['election_year'].unique())[0])
    tmp_election_dict['election_year']            = election_year

    dem_candidate                                 = str(single_election_df['dem_nominee'].unique()[0]) #getDemCandidate(single_election_df)[0])
    print(f"type: {type(dem_candidate)}")
    tmp_election_dict['dem_candidate']            = str(dem_candidate)
    
    rep_candidate                                 = str(single_election_df['rep_nominee'].unique()[0]) #getRepCandidate(single_election_df)
    tmp_election_dict['rep_candidate']            = str(rep_candidate)
    
    if (nationalVoteTotalDem > nationalVoteTotalRep):
        winning_party = 'democrat'
        winning_candidate = dem_candidate 
    else:
        winning_party = 'republican'
        winning_candidate = rep_candidate
        
    tmp_election_dict['winning_party']            = winning_party
    tmp_election_dict['winning_candidate']        = winning_candidate
    
    tmp_election_dict['total_national_dem_votes'] = nationalVoteTotalDem
    
    tmp_election_dict['total_national_rep_votes'] = nationalVoteTotalRep
    
    list_of_states = single_election_df['state'].unique().tolist()

    tmp_election_dict['states'] = {}
    for state in list_of_states:
        tmp_state_dict = {}           
        this_state_df = single_election_df[single_election_df['state'] == state]
        tmp_state_dict['state_name']     = this_state_df['state']
        state_dem_vote                   = this_state_df['dem_votes'].sum()
        state_rep_vote                   = this_state_df['rep_votes'].sum()
        tmp_state_dict['state_name']     = state
        tmp_state_dict['state_dem_vote'] = state_dem_vote
        tmp_state_dict['state_rep_vote'] = state_rep_vote
        
        if (state_rep_vote > state_dem_vote):
            tmp_state_dict['state_winning_party']     = 'republican'
            tmp_state_dict['state_winning_candidate'] = rep_candidate
        else:
            tmp_state_dict['state_winning_party']     = 'democrat'
            tmp_state_dict['state_winning_candidate'] = dem_candidate

        tmp_election_dict['states'][state] = tmp_state_dict
    
        # get list of counties for current state
        list_of_counties = this_state_df['county_name'].unique().tolist()
        tmp_state_dict['counties']={}
        for county in list_of_counties:
            tmp_county_dict = {}

            tmp_county_dict['county_name']     = county
            this_fips                          = this_state_df.loc[this_state_df['county_name'] == county, 'fips']
            fips                               = this_fips.iloc[0]
            tmp_county_dict['fips'] = fips
            
            this_county_dem_vote               = this_state_df.loc[this_state_df['county_name'] == county, 'dem_votes']
            county_dem_vote                    = this_county_dem_vote.iloc[0]           
            tmp_county_dict['county_dem_vote'] = county_dem_vote
           
            this_county_rep_vote               = this_state_df.loc[this_state_df['county_name'] == county, 'rep_votes']
            county_rep_vote                    = this_county_rep_vote.iloc[0]
            tmp_county_dict['county_rep_vote'] = county_rep_vote
       
            if (county_rep_vote > county_dem_vote):
                tmp_county_dict['county_winning_party']     = 'republican'
                tmp_county_dict['county_winning_candidate'] = rep_candidate
            else:
                tmp_county_dict['state_winning_party']     = 'democrat'
                tmp_county_dict['state_winning_candidate'] = dem_candidate
                
            tmp_state_dict['counties'][county]= tmp_county_dict

        tmp_election_dict['states'][state] = tmp_state_dict

    elections[election_year] = tmp_election_dict


elections_json_object = json.dumps(elections,default=default, indent = 4) # defaault = default
#print(elections_json_object)
with open("Data/historic_election_deep_mongo.json", "w") as outfile: 
    outfile.write(elections_json_object)

type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
type: <class 'str'>
